# Get from Address to Model

In [ ]:
import urllib
import requests
import json

## Geo-Admin API
Access of the API

In [ ]:
#Definition of parameters
text = input("Please enter an address: ") #Input of the address to search for
base_url="https://api3.geo.admin.ch/rest/services/api/SearchServer?"
parameters = {"searchText": text, "origins": "address", "type": "locations",}

#Request
r = requests.get(f"{base_url}{urllib.parse.urlencode(parameters)}")

data = json.loads(r.content)
#print(data)

Geographic display of returned results

In [ ]:
import folium

# Create a map centered at the first item's latitude and longitude
map_center = [data['results'][0]['attrs']['lat'], data['results'][0]['attrs']['lon']]
m = folium.Map(location=map_center, zoom_start=20)

# Add markers for each item with annotations
for i, result in enumerate(data['results']):
    lat = result['attrs']['lat']
    lon = result['attrs']['lon']
    label = f"Index: {i}"
    folium.Marker([lat, lon], popup=label).add_to(m)

# Display the map
m


In [ ]:
#Select the desired item
index = int(input("Please enter the index of the item you want to select: "))
selected_building = data['results'][index]
#print(selected_building)


## KML files
Check KML files for desired building

In [ ]:
import os
import re
import math

# Define the folder path
folder_path = "swissBUILDINGS3D"

# Get the latitude and longitude from the selected_item
lat = float(selected_building['attrs']['lat'])
lon = float(selected_building['attrs']['lon'])
print(f"Latitude: {lat}", f"Longitude: {lon}")

# Define the calculate_distance function
def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert coordinates to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = 6371 * c  # Earth radius in kilometers

    return distance

# Search for KML files in the folder
kml_files = []
for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".kml"):
            kml_files.append(os.path.join(root, file))

print(f"Found {len(kml_files)} KML files", kml_files)

# Define the tolerance in kilometers
tolerance = 0.01

# List to store the filtered KML data
filtered_kml_data = []

# Iterate through the KML files
for kml_file in kml_files:
    with open(kml_file, 'r') as file:
        kml_content = file.read()
        
        # Extract the latitude and longitude values from the KML content
        latitudes = re.findall(r'<latitude>(.*?)</latitude>', kml_content)
        longitudes = re.findall(r'<longitude>(.*?)</longitude>', kml_content)
        hrefs = re.findall(r'<href>(.*?)</href>', kml_content)
        
        # Convert the latitude and longitude values to float
        latitudes = [float(lat) for lat in latitudes]
        longitudes = [float(lon) for lon in longitudes]
        
        # Check if any entry is within the tolerance
        for i in range(len(latitudes)):
            distance = calculate_distance(lat, lon, latitudes[i], longitudes[i])
            if distance <= tolerance:
                filtered_kml_data.append((kml_file, latitudes[i], longitudes[i],  hrefs[i]))

# Print the filtered KML data
#for data in filtered_kml_data:
    #print(f"KML File: {data[0]}, Latitude: {data[1]}, Longitude: {data[2]}, href: {data[3]}")



## Get model data from kml

In [ ]:
import collada

kml = filtered_kml_data[0][0].split("\\")[1]
model = filtered_kml_data[0][3].split("/")[-1]

for data in filtered_kml_data:
    kml_file = data[0]
    lat = data[1]
    lon = data[2]
    href = data[3]
    
    # Load the COLLADA file
    coll = collada.Collada("swissBUILDINGS3D\\"+kml+"\\models\\"+model)
    #print ("swissBUILDINGS3D\\"+kml+"\\models\\"+model)
    # Extract the geometry
    #print(coll)
    #print(coll.geometries)
    #print(coll.geometries[0].primitives)
    #for i in coll.geometries[0].primitives[0]:
        #print(i)


In [ ]:
import numpy as np
from ipygany import PolyMesh

meshes=[]

for i in range(len(coll.geometries)):
    
    vertices = []
    faces = []

    for triangle in coll.geometries[i].primitives[0]:
        # Extract the vertices of the triangle
        v1, v2, v3 = triangle.vertices
        
        # Add the vertices to the vertices list
        vertices.append(v1)
        vertices.append(v2)
        vertices.append(v3)
        
        # Calculate the indices of the vertices in the faces list
        index1 = len(vertices) - 3
        index2 = len(vertices) - 2
        index3 = len(vertices) - 1
        
        # Add the indices to the faces list
        faces.append([index1, index2, index3])

    # Convert the vertices and faces lists to numpy arrays
    vertices = np.array(vertices)
    faces = np.array(faces)

    # Create the PolyMesh
    mesh = PolyMesh(vertices=vertices, triangle_indices=faces)
    meshes.append(mesh)

#print(meshes[0].vertices)


In [ ]:
#Draw the whole building
from ipygany import Scene

sceneHouse = Scene(meshes)
sceneHouse

In [ ]:
walls = [mesh for mesh in meshes if (mesh.vertices[2::3].max()- mesh.vertices[2::3].min())> 5]

sceneWalls = Scene(walls)
sceneWalls


In [ ]:
roof = [mesh for mesh in meshes if not (mesh.vertices[2::3].max()- mesh.vertices[2::3].min())> 5]

sceneRoof = Scene(roof)
sceneRoof

In [ ]:
num_triangles = len(walls[0].vertices)/9
ns_walls, we_walls = [], []
ns_vertices = []
ns_faces = []
we_vertices = []
we_faces = []
triangles = []
ns_index=0
we_index=0
for i in range(int(num_triangles)):
    triangle= walls[0].vertices[i*9:(i+1)*9]
    triangles.append(triangle)
for triangle in triangles:
    x1, y1, z1 = triangle[0], triangle[1], triangle[2]
    x2, y2, z2 = triangle[3], triangle[4], triangle[5]
    x3, y3, z3 = triangle[6], triangle[7], triangle[8]

    dir12 = np.array([abs(x2 - x1), abs(y2 - y1)])
    dir13 = np.array([abs(x3 - x1), abs(y3 - y1)])
    dir23 = np.array([abs(x3 - x2), abs(y3 - y2)])

    if(max(dir12[0], dir13[0], dir23[0])>max(dir12[1],dir13[1],dir23[1])):
        ns_vertices.append(triangle)
        ns_faces.append([ns_index, ns_index+1, ns_index+2])
        ns_index+=3

    else:
        we_vertices.append(triangle)
        we_faces.append([we_index, we_index+1, we_index+2])
        we_index+=3

ns_walls.append(PolyMesh(vertices=ns_vertices, triangle_indices=ns_faces))
we_walls.append(PolyMesh(vertices=we_vertices, triangle_indices=we_faces))

sceneNS =Scene(we_walls)
sceneNS


## Upload images from the building